# Auto Scheduling Engineering Week

## Computations

In [1]:
import numpy as np
from ortools.linear_solver import pywraplp

sessions = ['saic', 'iridium', 'mantech', 'xelevate', 'lcps', 'bwxt', 
            'salesforce', 'mpr', 'dss', 'new ag', 'usgs', 'terracon']

preferences = np.array([3.0,2,8,7,1,2,4,3,2,1,3,2])
preferences /= np.sum(preferences)
def generate_affinity_matrix(I, J, A):
    """Generate a binary matrix G[I, J] where each row has exactly A ones."""
    G = np.zeros((I, J), dtype=int)
    for i in range(I):
        ones_positions = np.random.choice(J, A, replace=False, p = preferences)  # Pick A unique columns
        G[i, ones_positions] = 1  # Set selected columns to 1
    return G


In [6]:
solver = pywraplp.Solver.CreateSolver('CBC')

# Define indices
I, J, K = 416, 12, 3  # Example sizes

G = generate_affinity_matrix(I,J,5)

RoomSize = 22
Auditorium_Size = 200

# Define binary variables X[i][j][k]
X = [[[solver.BoolVar(f'X_{i}_{j}_{k}') for k in range(K)] for j in range(J)] for i in range(I)]
print("Room Size = ", RoomSize)
print("Sign ups = ", np.sum(G, axis=0))

Room Size =  22
Sign ups =  [183 139 338 292  77 118 219 201 137  61 192 123]


In [7]:
# Add constraint: for every i, sum over all j, k must equal 3
for i in range(I):
    solver.Add(sum(G[i,j] * X[i][j][k] for j in range(J) for k in range(K)) == 3)

# 1 session for every student-block
for i in range(I):
    for k in range(K):
        solver.Add(sum(X[i][j][k] for j in range(J)) == 1)

# <= 1 block for every student-session
for i in range(I):
    for j in range(J):
        solver.Add(sum(X[i][j][k] for k in range(K)) <= 1)

Small_Rooms = [0,1,4,5,6,7,8,9,10,11]
Big_Rooms = [2,3]

# No room overfill
for j in Small_Rooms:
    for k in range(K):
        solver.Add(sum(X[i][j][k] for i in range(I)) <= RoomSize)
for j in Big_Rooms:
    for k in range(K):
        solver.Add(sum(X[i][j][k] for i in range(I)) <= Auditorium_Size)

# Every student is in a session each block
for k in range(K):
    solver.Add(sum(X[i][j][k] for i in range(I) for j in range(J)) == I)

In [8]:

# Example objective (can be anything)
solver.Maximize(sum(X[i][j][k] for i in range(I) for j in range(J) for k in range(K)))

# Solve
status = solver.Solve()

# Output results
if status == solver.OPTIMAL or status == solver.FEASIBLE:
    print("Solution found!\n")
    for i in range(1):
        print(f"Table for i = {i}:")
        print("    " + "   ".join(f"k={k}" for k in range(K)))  # Header row
        print("  " + "-" * (5 * K))  # Formatting line
        for j in range(J):
            row_values = "   ".join(str(int(X[i][j][k].solution_value())) for k in range(K))
            print(f"j={j} | {row_values}")  # Print row
        print("\n")  # Space between tables

Solution found!

Table for i = 0:
    k=0   k=1   k=2
  ---------------
j=0 | 0   0   0
j=1 | 0   0   0
j=2 | 1   0   0
j=3 | 0   0   1
j=4 | 0   1   0
j=5 | 0   0   0
j=6 | 0   0   0
j=7 | 0   0   0
j=8 | 0   0   0
j=9 | 0   0   0
j=10 | 0   0   0
j=11 | 0   0   0




In [9]:
if status == solver.OPTIMAL:
    print("Optimal solution found!")
elif status == solver.FEASIBLE:
    print("Feasible solution found (not necessarily optimal).")
elif status == solver.INFEASIBLE:
    print("No feasible solution exists.")
elif status == solver.UNBOUNDED:
    print("The problem is unbounded.")
elif status == solver.ABNORMAL:
    print("An abnormal error occurred.")
elif status == solver.NOT_SOLVED:
    print("Solver was not executed.")
else:
    print("Unknown status.")


Optimal solution found!


In [32]:
if status == solver.OPTIMAL or status == solver.FEASIBLE:
    X_solution = np.zeros((I, J, K), dtype=int)  # Initialize empty numpy array
    
    for i in range(I):
        for j in range(J):
            for k in range(K):
                X_solution[i, j, k] = int(X[i][j][k].solution_value())  # Store solution

## Results

413 students randomly selected 5 sessions from a list of 12. Sessions were not uniformly seeded -- some were more popular, others less.

In [40]:
print("Sessions offered: ", sessions)

Sessions offered:  ['saic', 'iridium', 'mantech', 'xelevate', 'lcps', 'bwxt', 'salesforce', 'mpr', 'dss', 'new ag', 'usgs', 'terracon']


In [42]:
print("Sign ups per session = ", np.sum(G, axis=0))

Sign ups per session =  [183 139 338 292  77 118 219 201 137  61 192 123]


Room size set at 22 per room, 200 per auditorium

An optimal solution was found assigning every student to a selected session and honoring room capacities. First we see the enrollment per session

In [33]:
for j in range(J):
    for k in range(K):
        print(f"Session {sessions[j]:15} block {k}   has {sum(int(X[i][j][k].solution_value()) for i in range(I))} attendees")

Session saic            block 0   has 22 attendees
Session saic            block 1   has 22 attendees
Session saic            block 2   has 22 attendees
Session iridium         block 0   has 22 attendees
Session iridium         block 1   has 22 attendees
Session iridium         block 2   has 22 attendees
Session mantech         block 0   has 102 attendees
Session mantech         block 1   has 124 attendees
Session mantech         block 2   has 105 attendees
Session xelevate        block 0   has 96 attendees
Session xelevate        block 1   has 80 attendees
Session xelevate        block 2   has 97 attendees
Session lcps            block 0   has 20 attendees
Session lcps            block 1   has 22 attendees
Session lcps            block 2   has 22 attendees
Session bwxt            block 0   has 22 attendees
Session bwxt            block 1   has 22 attendees
Session bwxt            block 2   has 22 attendees
Session salesforce      block 0   has 22 attendees
Session salesforce      bloc

And verify some sums

In [46]:
print("Total enrolled per block")
np.sum(X_solution,axis=0)

Total enrolled per block


array([[ 22,  22,  22],
       [ 22,  22,  22],
       [102, 124, 105],
       [ 96,  80,  97],
       [ 20,  22,  22],
       [ 22,  22,  22],
       [ 22,  22,  22],
       [ 22,  22,  22],
       [ 22,  22,  22],
       [ 22,  14,  16],
       [ 22,  22,  22],
       [ 22,  22,  22]])

In [47]:
print("Total attendees per session")
np.sum(np.sum(X_solution,axis=2),axis=0)

Total attendees per session


array([ 66,  66, 331, 273,  64,  66,  66,  66,  66,  52,  66,  66])

In [48]:
print("Total sessions per student")
np.sum(np.sum(X_solution,axis=2),axis=1)

Total sessions per student


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [49]:
print("Total students assigned per block")
np.sum(np.sum(X_solution,axis=1),axis=0)

Total students assigned per block


array([416, 416, 416])

Now just mathematically verify the constraints were satisfied

In [34]:
# Verify 1 session for every student-block
for i in range(I):
    for k in range(K):
        if sum(X[i][j][k] for j in range(J)).solution_value()<1:
            print(i,j,k )

In [35]:
# Verify 3 sessions for every studeny
for i in range(I):
    if(sum(X[i][j][k].solution_value() for j in range(J) for k in range(K)) < 3):
        print(i, sum(X[i][j][k].solution_value() for j in range(J) for k in range(K)))

And print the whole schedule

In [36]:
for i in range(I):
    s = f"Student #{i:04} "
    for k in range(K):
        for j in range(J):
            if X[i][j][k].solution_value() > 0:
                s += f" {sessions[j]:12} "
    print(s)

Student #0000  mantech       lcps          xelevate     
Student #0001  terracon      xelevate      mpr          
Student #0002  mantech       mpr           xelevate     
Student #0003  dss           mantech       xelevate     
Student #0004  xelevate      terracon      mpr          
Student #0005  xelevate      new ag        usgs         
Student #0006  salesforce    mantech       mpr          
Student #0007  mantech       xelevate      terracon     
Student #0008  mantech       saic          xelevate     
Student #0009  mantech       iridium       xelevate     
Student #0010  mantech       saic          xelevate     
Student #0011  mantech       saic          mpr          
Student #0012  mantech       saic          xelevate     
Student #0013  mantech       new ag        xelevate     
Student #0014  mantech       iridium       xelevate     
Student #0015  xelevate      new ag        mantech      
Student #0016  xelevate      mantech       lcps         
Student #0017  xelevate      ma